In [1]:
#Importing necessary libraries
import findspark
findspark.init()
findspark.find()
from pyspark.sql import DataFrame, SparkSession

In [2]:
#Creating spark session
spark = SparkSession \
       .builder \
       .appName("IoT Analysis") \
       .getOrCreate()

spark

23/02/09 18:07:45 WARN Utils: Your hostname, subbu resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/02/09 18:07:45 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/09 18:07:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Read the data into a Dataframe.
df=spark.read.json("iot_devices.json")

In [4]:
#Convert the Dataframe into a temporary view called iot.
df.createOrReplaceTempView("iot")

In [13]:
#selecting the data from iot view
spark.sql("select * from iot").show(5, False)

+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
|battery_level|c02_level|cca2|cca3|cn           |device_id|device_name          |humidity|ip           |latitude|lcd   |longitude|scale  |temp|timestamp    |
+-------------+---------+----+----+-------------+---------+---------------------+--------+-------------+--------+------+---------+-------+----+-------------+
|8            |868      |US  |USA |United States|1        |meter-gauge-1xbYRYcj |51      |68.161.225.1 |38.0    |green |-97.0    |Celsius|34  |1458444054093|
|7            |1473     |NO  |NOR |Norway       |2        |sensor-pad-2n2Pea    |70      |213.161.254.1|62.47   |red   |6.15     |Celsius|11  |1458444054119|
|2            |1556     |IT  |ITA |Italy        |3        |device-mac-36TWSKiT  |44      |88.36.5.1    |42.83   |red   |12.83    |Celsius|19  |1458444054120|
|6            |1080     |US  |USA |United States|4  

In [14]:
#Count how many devices are there from each country and display the output.
spark.sql("select count(distinct device_id) as device_id,cca3 from iot group by cca3 order by device_id desc").show()

+---------+----+
|device_id|cca3|
+---------+----+
|    70405| USA|
|    14455| CHN|
|    12100| JPN|
|    11879| KOR|
|     7942| DEU|
|     6486| GBR|
|     6041| CAN|
|     5989| RUS|
|     5305| FRA|
|     3224| BRA|
|     3119| AUS|
|     2915| ITA|
|     2880| SWE|
|     2744| POL|
|     2488| NLD|
|     2310| ESP|
|     2128| TWN|
|     1867| IND|
|     1507| CZE|
|     1487| NOR|
+---------+----+
only showing top 20 rows



In [15]:
#Display all the countries whose carbon dioxide level is more than 1400. Sort the output in descending order.
spark.sql("select cca3, c02_level from iot where c02_level > 1400 order by c02_level desc").show()

+----+---------+
|cca3|c02_level|
+----+---------+
| BGR|     1599|
| USA|     1599|
| EGY|     1599|
| TWN|     1599|
| DEU|     1599|
| DEU|     1599|
| CAN|     1599|
| JPN|     1599|
| DEU|     1599|
| BGR|     1599|
| ARG|     1599|
| GBR|     1599|
| USA|     1599|
| USA|     1599|
| USA|     1599|
| CHN|     1599|
| KOR|     1599|
| USA|     1599|
| NOR|     1599|
| POL|     1599|
+----+---------+
only showing top 20 rows



In [16]:
#Select all countries' devices with high-levels of C02 and group by cca3 and order by device_ids (Hint: For high CO2 level, the LCD status will be RED).
spark.sql("select cca3, count(distinct device_id) as device_id from iot where lcd == 'red' group by cca3 order by device_id desc").show()

+----+---------+
|cca3|device_id|
+----+---------+
| USA|    17489|
| CHN|     3616|
| KOR|     2942|
| JPN|     2935|
| DEU|     1966|
| GBR|     1660|
| CAN|     1564|
| RUS|     1508|
| FRA|     1353|
| BRA|      856|
| AUS|      769|
| SWE|      724|
| ITA|      713|
| POL|      664|
| NLD|      646|
| ESP|      586|
| TWN|      542|
| IND|      446|
| NOR|      399|
| UKR|      373|
+----+---------+
only showing top 20 rows



In [17]:
#find out all devices in countries whose batteries need replacements.Assuming battery level 0 means need replacement
spark.sql("select cca3, count(distinct device_id) as device_id from iot where battery_level == 0 group by cca3 order by device_id desc").show()

+----+---------+
|cca3|device_id|
+----+---------+
| USA|     7043|
| CHN|     1415|
| KOR|     1217|
| JPN|     1210|
| DEU|      760|
| GBR|      650|
| CAN|      612|
| RUS|      600|
| FRA|      582|
| BRA|      374|
| AUS|      322|
| SWE|      293|
| ITA|      287|
| POL|      278|
| NLD|      251|
| ESP|      223|
| TWN|      207|
| IND|      189|
| UKR|      149|
| HKG|      149|
+----+---------+
only showing top 20 rows

